In [2]:
import csv
import tkinter as tk
from tkinter import ttk
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import json
import datetime

current_date = datetime.datetime.today()
day_before = current_date - datetime.timedelta(days=1)
formatted_date = day_before.strftime("%Y-%m-%d")

# Define the function to be called when the button is clicked
def button_click():
    # Get the user input from the text box
    resort = resort_box.get()
    country = country_box.get()
    user_input = f"{resort}, {country}"
    output_label.config(text=f"You selected: {user_input}!\n Weather info in the next tab")
    
    global LOCATION
    
    import requests

    API_KEY = '582cb3d85cc48da815cdb43fa5a6f86f'
    LOCATION = user_input
    # Make a GET request to the API endpoint for current weather data
    response = requests.get(f'http://api.weatherstack.com/forecast?access_key={API_KEY}&query={LOCATION}')
    if response.status_code == 200:
        # Parse the response data as JSON
        data = response.json()
        # Extract the relevant weather data from the JSON data
        temperature = data['current']['temperature']
        weather_description = data['current']['weather_descriptions']
        location_name = data['location']['name']
        country = data['location']['country']
        region = data['location']['region']
        latitude = data['location']['lat']
        longitude = data['location']['lon']
        timezone_id = data['location']['timezone_id']
        localtime = data['location']['localtime']
        observation_time = data['current']['observation_time']
        weather_code = data['current']['weather_code']
        weather_icon = data['current']['weather_icons'][0]
        wind_speed = data['current']['wind_speed']
        wind_degree = data['current']['wind_degree']
        wind_direction = data['current']['wind_dir']
        pressure = data['current']['pressure']
        precip = data['current']['precip']
        humidity = data['current']['humidity']
        cloud_cover = data['current']['cloudcover']
        feels_like = data['current']['feelslike']
        uv_index = data['current']['uv_index']
        visibility = data['current']['visibility']
        is_day = data['current']['is_day']
        forecast_date = data['forecast'][formatted_date]['date']
        forecast_date_epoch = data['forecast'][formatted_date]['date_epoch']
        sunrise_time = data['forecast'][formatted_date]['astro']['sunrise']
        sunset_time = data['forecast'][formatted_date]['astro']['sunset']
        moonrise_time = data['forecast'][formatted_date]['astro']['moonrise']
        moonset_time = data['forecast'][formatted_date]['astro']['moonset']
        moon_phase = data['forecast'][formatted_date]['astro']['moon_phase']
        moon_illumination = data['forecast'][formatted_date]['astro']['moon_illumination']
        min_temp = data['forecast'][formatted_date]['mintemp']
        max_temp = data['forecast'][formatted_date]['maxtemp']
        avg_temp = data['forecast'][formatted_date]['avgtemp']
        total_snow = data['forecast'][formatted_date]['totalsnow']
        sun_hour = data['forecast'][formatted_date]['sunhour']
        uv_index_forecast = data['forecast'][formatted_date]['uv_index']

        # Create a Pandas dataframe to hold the weather data
        df = pd.DataFrame({
            'Location': [location_name],
            'Country': [country],
            'Region': [region],
            'Weather Description': [weather_description],
            'Temperature': [temperature],
            'Feels Like': [feels_like],
            'Min Temperature': [min_temp],
            'Max Temperature': [max_temp],
            'Cloud Cover': [cloud_cover],
            'Total Snow': [total_snow],
            'Precipitation': [precip],
            'Wind Speed': [wind_speed],
            'Wind Direction': [wind_direction],
            'Humidity': [humidity],
            'Pressure': [pressure],
            'Visibility': [visibility],
            'UV Index': [uv_index],
            'Local Time': [localtime],
            'Sunrise Time': [sunrise_time],
            'Sunset Time': [sunset_time]
            })
    df_transposed = df.T
    df_transposed = df_transposed.reset_index()

    global tab2
    if tab2 is None:
        # Create the tab2
        tab2 = ttk.Frame(tab_control)
        tab_control.add(tab2, text="Information")

    # Create a table and histogram in the second tab
    table = ttk.Treeview(tab2, columns=['Value'], height=20)

    # Add headings for the treeview table
    table.heading('#0', text='Metrics')
    table.heading('Value', text='Info')

    # Add data to the treeview table
    for index, row in df.iterrows():
        for column in df.columns:
            table.insert("", "end", text=column, values=[row[column]])

    # Pack the treeview table into the tkinter window
    table.pack(side='left', padx=10, pady=10)
     
    def sun_estimation(data):
        sun_score = 0
       
        # Temperature score
        if temperature >= 30:
            sun_score += 20
        elif temperature >= 20:
            sun_score += 15
        elif temperature >= 10:
            sun_score += 5
        elif temperature >= 0:
            sun_score += 1
        
        # Humidity score
        if humidity <= 50:
            sun_score += 20
        elif humidity <= 70:
            sun_score += 5
        else:
            sun_score += 1
        
        # Cloud cover score
        if cloud_cover <= 10:
            sun_score += 30
        elif cloud_cover <= 30:
            sun_score += 5
        else:
            sun_score += 1
        
        # UV index score
        if uv_index >= 8:
            sun_score += 30
        elif uv_index >= 5:
            sun_score += 15
        elif uv_index >= 3:
            sun_score += 5
        else:
            sun_score += 1
            
        return sun_score

    sun_score = sun_estimation(data)
    def rain_estimation(data):
        rain_score = 0
              
        # Humidity score
        if humidity >= 80:
            rain_score += 30
        elif humidity >= 70:
            rain_score += 10
        else:
            rain_score += 1
        
        # Precipitation score
        if precip > 2:
            rain_score += 50
        elif precip > 0.5:
            rain_score += 10
        elif precip > 0:
            rain_score += 5
        else:
            rain_score += 0
        
        # Cloud cover score
        if cloud_cover >= 70:
            rain_score += 10
        elif cloud_cover >= 50:
            rain_score += 5
        else:
            rain_score += 0
            
            # Temperature score
        if temperature >= 25:
            rain_score += 0
        elif temperature >= 20:
            rain_score += 1
        elif temperature >= 15:
            rain_score += 5
        elif temperature >= 10:
            rain_score += 10
        elif temperature < 0:
            rain_score += 0
        else:
            rain_score += 10
            
        return rain_score

    rain_score = rain_estimation(data)
    def snow_estimation(data):
        snow_score = 0

        # Humidity score
        if humidity >= 90:
            snow_score += 15
        elif humidity >= 80:
            snow_score += 0
        
        # Precipitation score
        if total_snow > 2:
            snow_score += 30
        elif total_snow > 0.5:
            snow_score += 20
        elif total_snow > 0:
            snow_score += 10

        # Temperature score
        if temperature <= -5:
            snow_score += 10
        elif temperature <= 0:
            snow_score += 10
        elif temperature <= 5:
            snow_score = 0  
        return snow_score

    snow_score = snow_estimation(data)

    def avalanche_estimation(data):
        avalanche_score = (snow_score + total_snow) / 2
        return avalanche_score

    avalanche_score = avalanche_estimation(data)

    scores = [sun_score, rain_score, snow_score, avalanche_score]
    labels = ['Sun', 'Rain', 'Snow', 'Avalanche']
    colors = ['yellow', 'blue', 'red', 'gray']

    # Create the figure and axis objects
    fig, ax = plt.subplots()

    # Create bar chart
    ax.bar(labels, scores, color=colors)

    # Add title and axis labels
    ax.set_title('Weather Estimates for Today')
    ax.set_xlabel('Weather Type')
    ax.set_ylabel('Probability')

    # Create the canvas and pack it into the tkinter window
    canvas = FigureCanvasTkAgg(fig, master=tab2)
    canvas.get_tk_widget().pack(side="right", padx=10, pady=10)

# Create the main window and set its properties
root = tk.Tk()
root.title("Ski Resort Selector")
root.geometry("1000x500")

# Create a tab control with two tabs
tab_control = ttk.Notebook(root)
tab1 = ttk.Frame(tab_control)
tab_control.add(tab1, text="Select Location")
tab_control.pack(expand=1, fill="both")

#Create an empty tab2 to be generated later
tab2 = None

# Create a list of countries and ski resorts
countries = []
ski_resorts = {}
with open('ski_resorts.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        country = row['Country']
        ski_resort = row['Name']
        if country not in countries:
            countries.append(country)
        if country in ski_resorts:
            ski_resorts[country].append(ski_resort)
        else:
            ski_resorts[country] = [ski_resort]

# Create the widgets
country_label = ttk.Label(tab1, text="Country:")
country_label.pack(side="left", padx=10, pady=10)
country_box = ttk.Combobox(tab1, values=countries)
country_box.pack(side="left", padx=10, pady=10)
country_box.current(0)
resort_label = ttk.Label(tab1, text="Ski Resort:")
resort_label.pack(side="left", padx=10, pady=10)
resort_box = ttk.Combobox(tab1)
resort_box.pack(side="left", padx=10, pady=10)

# Function to update the ski resorts list based on the selected country
def update_resorts(*args):
    country = country_box.get()
    resorts = ski_resorts.get(country, [])
    resort_box.config(values=resorts)
    resort_box.current(0)

# Bind the update_resorts function to the country_box
country_box.bind("<<ComboboxSelected>>", update_resorts)

# Call the update_resorts function to update the ski resorts list based on the default country
update_resorts()

# Create a button in the first tab that calls the button_click function
button = ttk.Button(tab1, text="Next", command=button_click)
button.pack(side="left", padx=10, pady=10)

# Create a label in the first tab to display the output
output_label = ttk.Label(tab1, text="")
output_label.pack(side="left", padx=10, pady=10)

# Select the tab2
tab_control.select(tab2)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\marco\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\marco\AppData\Local\Temp\ipykernel_18652\531092632.py", line 57, in button_click
    forecast_date = data['forecast'][formatted_date]['date']
KeyError: '2023-05-14'
